## Setup GOLDILOX Insight's Share space

In [ ]:
-- 1) Create the database and schema
CREATE DATABASE IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE;
CREATE SCHEMA   IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES;

-- Copy Table data
CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.QUERY_HISTORY AS
    SELECT * FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.QUERY_HISTORY WHERE START_TIME > DATEADD('MONTH', -3, CURRENT_DATE());

CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.RECOMMENDATION_RESULT AS
    SELECT * FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.RECOMMENDATION_RESULT;

-- 2) Create a direct share and add the table
CREATE SHARE IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE;

-- Grant USAGE on the database that CONTAINS the secure view
GRANT USAGE ON DATABASE GOLDILOX_INSIGHTS_SHARE TO SHARE GOLDILOX_INSIGHTS_SHARE;
GRANT USAGE ON SCHEMA   GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES TO SHARE GOLDILOX_INSIGHTS_SHARE;

GRANT SELECT ON ALL TABLES IN SCHEMA GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES   TO SHARE GOLDILOX_INSIGHTS_SHARE;

-- 3) Add your provider account to the share
ALTER SHARE GOLDILOX_INSIGHTS_SHARE ADD ACCOUNTS = SEFURGX.ALB16562;

## Create a task to update daily

In [ ]:
CREATE SCHEMA IF NOT EXISTS GOLDILOX_INSIGHTS_SHARE.TASKS;
CREATE OR REPLACE TASK GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA
  WAREHOUSE = GOLDILOX_APP_CLIENT_WH
  SCHEDULE = 'USING CRON 30 06 * * * UTC'
  COMMENT = 'Rebuild shared tables daily'
AS
BEGIN
  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.QUERY_HISTORY
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.QUERY_HISTORY
  WHERE START_TIME > DATEADD('MONTH', -1, CURRENT_DATE());

  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.RECOMMENDATION_RESULT
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.RECOMMENDATION_RESULT;

  CREATE OR REPLACE TABLE GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES.TRAINING_DATA
  COPY GRANTS
  AS
  SELECT *
  FROM GOLDILOX_INSIGHTS.SHARED_VIEWS.TRAINING_DATA;

  GRANT SELECT ON ALL TABLES IN SCHEMA GOLDILOX_INSIGHTS_SHARE.SHARED_TABLES   TO SHARE GOLDILOX_INSIGHTS_SHARE;
END;

ALTER TASK GOLDILOX_INSIGHTS_SHARE.TASKS.REFRESH_DATA RESUME;
